# Import Library

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress info and warning messages

In [2]:
import tables
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import (
    Sequential,
    Model
)
from tensorflow.keras.layers import (
    Dense,
    LeakyReLU,
    Dropout,
    BatchNormalization,
    Reshape,
    Flatten,
    Input, 
    Embedding, 
    multiply,
    Concatenate
)
from tensorflow.keras.optimizers import Adam

2025-01-14 14:59:40.815010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736841580.825112 1394368 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736841580.827930 1394368 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Preprocessing Dataset

In [5]:
# iot_data = pd.read_csv('../Dataset/iot23_combinednew.csv')
iot_data = pd.read_hdf('../Dataset/iot23_combined.h5')

iot_data

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1525879831.045045,C3GBTkINvXNjVGtN5,192.168.100.103,41101.0,111.40.23.49,23.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,60.0,0.0,0.0,(empty) Malicious PartOfAHorizontalPortScan
1,1525879832.01624,CDe43c1PtgynajGI6,192.168.100.103,60905.0,131.174.215.147,23.0,tcp,-,2.998796,0,...,S0,-,-,0.0,S,3.0,180.0,0.0,0.0,(empty) Malicious PartOfAHorizontalPortScan
2,1525879832.024985,CJaDcG3MZzvf1YVYI4,192.168.100.103,44301.0,91.42.47.63,23.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,60.0,0.0,0.0,(empty) Malicious PartOfAHorizontalPortScan
3,1525879832.044975,CMBrup3BLXivSp4Avc,192.168.100.103,50244.0,120.210.108.200,23.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,60.0,0.0,0.0,(empty) Malicious PartOfAHorizontalPortScan
4,1525879833.016171,CfHl9r3XMYtDQRrHnh,192.168.100.103,34243.0,147.7.65.203,49560.0,tcp,-,2.998804,0,...,S0,-,-,0.0,S,3.0,180.0,0.0,0.0,(empty) Benign -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446625,1562164752.145405,CbiAr234EfpzQInxbg,0.0.0.0,68.0,255.255.255.255,67.0,udp,dhcp,90.034713,3300.0,...,S0,-,-,0.0,D,11.0,3608.0,0.0,0.0,- benign -
1446626,1562164752.076456,CqEhZf1lYmVokqECn3,::,143.0,ff02::16,0.0,icmp,-,90.39997,340.0,...,OTH,-,-,0.0,-,9.0,844.0,0.0,0.0,- benign -
1446627,1562164752.588413,Cr5vKO1CGfMQwFp22c,::,135.0,ff02::1:ffd5:cdf,136.0,icmp,-,89.82403,72.0,...,OTH,-,-,0.0,-,3.0,216.0,0.0,0.0,- benign -
1446628,1562164753.612461,CQ5cJ21U9NUbUWIznd,fe80::5bcc:698e:39d5:cdf,143.0,ff02::16,0.0,icmp,-,45.215915,200.0,...,OTH,-,-,0.0,-,8.0,648.0,0.0,0.0,- benign -


In [ ]:
# Menghapus kolom 'Unnamed: 0'
iot_data = iot_data.drop(columns=['Unnamed: 0'])

iot_data

In [ ]:
# Define a mapping for the new labels
label_mapping = {
    'PartOfAHorizontalPortScan': 'Scanning',
    'DDoS': 'DDoS',
    'C&C-Torii': 'Botnet',
    'Benign': 'Benign',
    'C&C': 'C&C',
    'C&C-HeartBeat-FileDownload': 'C&C',
    'Attack': 'Attack'
}

# Apply the mapping to the 'label' column
iot_data['label'] = iot_data['label'].map(label_mapping)

# Check the unique labels after transformation
unique_labels = iot_data['label'].unique()
print("Labels setelah transformasi:", unique_labels)

In [ ]:
# Menampilkan kolom
print("Kolom:", iot_data.columns.tolist())

In [ ]:
# Menampilkan kolom
print(iot_data["label"])

In [ ]:
# tampilkan ada label apa aja dari kolom label
unique_labels = iot_data["label"].unique()
print(unique_labels)

In [ ]:
iot_data.info()

In [ ]:
iot_data.isna().sum()

In [ ]:
# Melihat distribusi missing values di antara label
print(iot_data[iot_data["duration"].isna()]["label"].value_counts())

In [ ]:
iot_data.nunique()

In [13]:
iot_data[["duration", "orig_bytes", "resp_bytes"]] = iot_data[
    ["duration", "orig_bytes", "resp_bytes"]
].fillna(0)

In [ ]:
iot_data.isna().sum()

In [ ]:
iot_data.tail(5)

In [ ]:
import matplotlib.pyplot as plt

label_counts = iot_data["label"].value_counts()
print(label_counts)

# Visualisasi distribusi label
plt.figure(figsize=(10, 5))
label_counts.plot(kind="bar")
plt.title("Distribusi Label dalam Dataset IoT")
plt.xlabel("Label")
plt.ylabel("Jumlah")
plt.show()

### Encode Label to Numeric and Normalisasi Fitur

In [17]:
label_encoder = LabelEncoder()

iot_data['label'] = label_encoder.fit_transform(iot_data['label'])

X = iot_data.drop('label', axis=1)
y = iot_data['label']

In [ ]:
X

In [ ]:
y

In [ ]:
unique_y = y.unique()
print(unique_y)

In [ ]:
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print(label_mapping)

In [22]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Generator

In [ ]:
print(X_train.shape[1]) # number of features or columns

In [ ]:
# Set dimensions based on your dataset
input_dim = X_train.shape[1]  # number of features

# Define the Generator
generator = Sequential([
    Dense(64, input_dim=100),  # input is random noise
    LeakyReLU(0.2),
    BatchNormalization(momentum=0.8),
    Dense(128),
    LeakyReLU(0.2),
    BatchNormalization(momentum=0.8),
    Dense(256),
    LeakyReLU(0.2),
    Dense(input_dim, activation='tanh')  # output matches feature dimension
])

generator.summary()


# Discriminator

In [ ]:
# Define the Discriminator
discriminator = Sequential([
    Dense(256, input_dim=input_dim),
    LeakyReLU(0.2),
    Dropout(0.3),
    Dense(128),
    LeakyReLU(0.2),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # binary output for real/fake
])

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.summary()


# Combining Generator and Discriminator (GAN Model)

In [ ]:
# Freeze discriminator weights when training the generator
discriminator.trainable = False

# GAN model
gan_input = Input(shape=(100,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)

gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')
gan.summary()


# Training the GAN

In [28]:
epochs = 10000
batch_size = 64
sample_interval = 1000  # interval to save generated samples for evaluation

In [ ]:
import tensorflow as tf

for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_data = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_data = generator.predict(noise)
    
    # Labels for real and fake data
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    
    # Train discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train Generator (where discriminator's weights are frozen)
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid_labels = np.ones((batch_size, 1))  # misleading labels for GAN training
    g_loss = gan.train_on_batch(noise, valid_labels)
    
    # Logging the progress
    if epoch % sample_interval == 0:
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")


In [30]:
# Jumlah sampel sintetik yang ingin Anda hasilkan untuk setiap kelas:
num_samples = {
    0: 8000,  # Attack
    1: 3000,  # Benign
    2: 8000,  # Botnet
    3: 2000,  # C&C
    4: 1000,  # DDoS
    5: 1000   # Scanning
}

synthetic_data_list = []
synthetic_labels_list = []

In [ ]:
print(num_samples.items())


In [ ]:
for class_label, n_samples in num_samples.items():
    # Membagi n_samples ke dalam beberapa batch untuk memastikan jumlah yang diinginkan
    synthetic_data_for_class = []
    batch_size = 1000  # Menentukan ukuran batch untuk efisiensi (misalnya 1000 sampel per batch)

    for _ in range(n_samples // batch_size):
        noise = np.random.normal(0, 1, (batch_size, 100))  # Sesuaikan dengan dimensi noise generator Anda
        synthetic_data_batch = generator.predict(noise)
        synthetic_data_for_class.append(synthetic_data_batch)

    # Jika n_samples bukan kelipatan batch_size, menghasilkan batch tambahan
    if n_samples % batch_size != 0:
        remaining_samples = n_samples % batch_size
        noise = np.random.normal(0, 1, (remaining_samples, 100))
        synthetic_data_batch = generator.predict(noise)
        synthetic_data_for_class.append(synthetic_data_batch)

    # Menggabungkan semua batch untuk kelas saat ini
    synthetic_data_for_class = np.vstack(synthetic_data_for_class)
    class_labels = np.full(n_samples, class_label)
    
    # Menyimpan data dan label sintetik untuk kelas saat ini
    synthetic_data_list.append(synthetic_data_for_class)
    synthetic_labels_list.append(class_labels)

# Menggabungkan semua data sintetik dari setiap kelas
synthetic_data = np.vstack(synthetic_data_list)
synthetic_labels = np.hstack(synthetic_labels_list)

# Verifikasi jumlah sampel untuk setiap kelas
from collections import Counter
print("Distribusi sampel sintetik:", Counter(synthetic_labels))

print("Shape of synthetic data:", synthetic_data_for_class.shape)


In [ ]:
# Menggabungkan semua data sintetik
synthetic_data = np.vstack(synthetic_data_list)
synthetic_labels = np.hstack(synthetic_labels_list)
                             
# Menggabungkan data sintetik dengan data asli
X_augmented = np.vstack((X_train, synthetic_data))
y_augmented = np.hstack((y_train, synthetic_labels))

# Melatih classifier (contoh: RandomForest)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_augmented, y_augmented)

# Evaluasi pada test set
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
print("Classifier Accuracy on Test Set:", accuracy_score(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot distribusi kelas pada data sintetik
plt.figure(figsize=(10, 5))
counts, bins, patches = plt.hist(synthetic_labels, bins=np.arange(len(np.unique(synthetic_labels))+1)-0.5, edgecolor='black', rwidth=0.8)

# Menambahkan jumlah kelas di atas setiap batang histogram
for i in range(len(counts)):
    plt.text(patches[i].get_x() + patches[i].get_width()/2, counts[i], int(counts[i]), 
             ha='center', va='bottom', fontsize=10)

plt.title("Distribusi Kelas pada Data Sintetik")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Sampel")
plt.xticks(np.arange(len(np.unique(synthetic_labels))))
plt.show()


In [ ]:
# Asumsikan `y_train` adalah label data asli Anda
plt.figure(figsize=(10, 5))

# Plot distribusi kelas pada data asli
plt.hist(y_train, bins=np.arange(len(np.unique(y_train))+1)-0.5, alpha=0.5, label="Data Asli", edgecolor='black', rwidth=0.8)

# Plot distribusi kelas pada data sintetik
plt.hist(synthetic_labels, bins=np.arange(len(np.unique(synthetic_labels))+1)-0.5, alpha=0.5, label="Data Sintetik", edgecolor='black', rwidth=0.8)

plt.title("Perbandingan Distribusi Kelas antara Data Asli dan Sintetik")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Sampel")
plt.xticks(np.arange(len(np.unique(synthetic_labels))))
plt.legend()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
# class_weights = {0: 1, 1: 1, 2: 10, 3: 1, 4: 1, 5: 1}
classifier = RandomForestClassifier(class_weight='balanced', 
                                    random_state=42, 
                                    max_depth=10, 
                                    min_samples_split=5, 
                                    min_samples_leaf=2)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, fbeta_score

# Predictions
y_pred = classifier.predict(X_test)

# Classification report for precision, recall, and F1 (macro and weighted)
report = classification_report(y_test, y_pred, zero_division=0)
print(report)


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred, pos_label=1)  # Gantilah dengan kelas yang lebih relevan
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
from collections import Counter
print("Class distribution in y_test:", Counter(y_test))
print("Class distribution in y_pred:", Counter(y_pred))


In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(
    use_label_encoder=False,  # Menonaktifkan label encoder yang sudah deprecated
    eval_metric='mlogloss',   # Menetapkan metric evaluasi untuk klasifikasi multikelas
    random_state=42,          # Mengatur seed untuk reprodusibilitas
    max_depth=10,             # Contoh pengaturan parameter untuk menghindari overfitting
    min_child_weight=1,       # Pengaturan minimum bobot anak
    learning_rate=0.1,        # Learning rate
    n_estimators=100          # Jumlah estimator (trees)
)

# Melatih model
model.fit(X_train, y_train)

# Prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model dengan akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Classifier Accuracy on Test Set: {accuracy:.4f}")

# Menampilkan classification report
report = classification_report(y_test, y_pred, zero_division=0)
print("Classification Report:\n", report)

# Menampilkan distribusi kelas di y_test dan y_pred untuk analisis lebih lanjut
print("Class distribution in y_test:", Counter(y_test))
print("Class distribution in y_pred:", Counter(y_pred))

# Jika ingin visualisasi Precision-Recall Curve untuk satu kelas (misalnya kelas 1)
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_test, y_pred, pos_label=1)  # Ubah jika perlu untuk kelas lainnya
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Balanced Random Forest
brf_model = BalancedRandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced_subsample',  # Menyeimbangkan kelas selama pembelajaran
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2
)

# Melatih model
brf_model.fit(X_train, y_train)

# Evaluasi pada test set
y_pred_brf = brf_model.predict(X_test)

# Laporan klasifikasi untuk precision, recall, dan F1
report_brf = classification_report(y_test, y_pred_brf, zero_division=0)
print("Classification Report for Balanced Random Forest:\n", report_brf)

# Akurasi model
accuracy_brf = accuracy_score(y_test, y_pred_brf)
print("Accuracy of Balanced Random Forest on Test Set: ", accuracy_brf)
